In [2]:
pip install mediapipe

                                              0.0/50.9 MB ? eta -:--:--
                                             0.0/50.9 MB 435.7 kB/s eta 0:01:57
                                             0.1/50.9 MB 656.4 kB/s eta 0:01:18
                                              0.2/50.9 MB 1.1 MB/s eta 0:00:45
                                              0.3/50.9 MB 1.4 MB/s eta 0:00:37
                                              0.5/50.9 MB 2.1 MB/s eta 0:00:24
                                              0.7/50.9 MB 2.8 MB/s eta 0:00:19
                                              1.0/50.9 MB 3.1 MB/s eta 0:00:17
     -                                        1.3/50.9 MB 3.4 MB/s eta 0:00:15
     -                                        1.6/50.9 MB 3.8 MB/s eta 0:00:13
     -                                        2.0/50.9 MB 4.2 MB/s eta 0:00:12
     -                                        2.3/50.9 MB 4.4 MB/s eta 0:00:11
     --                                       2.6/50.9 MB

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     -----------------------                  26.1/44.8 MB 2.9 MB/s eta 0:00:07
     -----------------------                  26.2/44.8 MB 2.9 MB/s eta 0:00:07
     -----------------------                  26.4/44.8 MB 2.9 MB/s eta 0:00:07
     -----------------------                  26.5/44.8 MB 2.9 MB/s eta 0:00:07
     -----------------------                  26.6/44.8 MB 2.9 MB/s eta 0:00:07
     -----------------------                  26.8/44.8 MB 3.0 MB/s eta 0:00:07
     ------------------------                 26.9/44.8 MB 3.0 MB/s eta 0:00:06
     ------------------------                 27.0/44.8 MB 3.0 MB/s eta 0:00:06
     ------------------------                 27.2/44.8 MB 3.0 MB/s eta 0:00:06
     ------------------------                 27.2/44.8 MB 2.9 MB/s eta 0:00:06
     ------------------------                 27.4/44.8 MB 3.0 MB/s eta 0:00:06
     ------------------------                 27.6/44.8 MB 3.0 MB/s eta 0:00:06
     ------------------------          

In [4]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import cv2
import mediapipe as mp
import tkinter as tk
from tkinter import ttk
import numpy as np

In [6]:


# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Initialize Tkinter GUI
root = tk.Tk()
root.title("Gesture-Based Interactive Interface")
root.geometry("400x600")

# Frame for scrollable area
frame_main = tk.Frame(root)
frame_main.pack(fill=tk.BOTH, expand=True)

# Adding Canvas for scrollable area
canvas = tk.Canvas(frame_main)
scrollbar = ttk.Scrollbar(frame_main, orient="vertical", command=canvas.yview)
scrollable_frame = ttk.Frame(canvas)

# Configure scroll behavior
scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)

canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)
canvas.pack(side="left", fill="both", expand=True)
scrollbar.pack(side="right", fill="y")

# Add interactive buttons in scrollable frame
buttons = []
for i in range(10):
    button = ttk.Button(scrollable_frame, text=f"Button {i+1}")
    button.pack(pady=5, padx=5)
    buttons.append(button)

# Functions to handle scroll and button actions
def scroll_up():
    canvas.yview_scroll(-1, "units")

def scroll_down():
    canvas.yview_scroll(1, "units")

def simulate_button_click(button_id):
    buttons[button_id].invoke()

# OpenCV Video Capture for Gesture Recognition
cap = cv2.VideoCapture(0)

def detect_gesture():
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Get coordinates of thumb and index finger for pinch detection
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            
            # Calculate pixel coordinates
            thumb_coords = (int(thumb_tip.x * frame.shape[1]), int(thumb_tip.y * frame.shape[0]))
            index_coords = (int(index_tip.x * frame.shape[1]), int(index_tip.y * frame.shape[0]))

            # Distance between thumb and index to detect pinch
            distance = np.sqrt((thumb_coords[0] - index_coords[0]) ** 2 + (thumb_coords[1] - index_coords[1]) ** 2)
            
            # Define gesture actions based on distance and hand position
            if distance < 30:
                # Pinch gesture detected - Simulate a button click
                simulate_button_click(0)  # Replace 0 with logic to detect which button to click

            # Use hand y-coordinate to control scrolling
            if thumb_coords[1] < frame.shape[0] // 3:
                scroll_up()
            elif thumb_coords[1] > 2 * frame.shape[0] // 3:
                scroll_down()
    
    # Display the frame
    cv2.imshow("Gesture Control", frame)
    cv2.waitKey(1)
    
    # Re-run this function after a short delay
    root.after(10, detect_gesture)

# Start gesture detection loop
root.after(10, detect_gesture)

# Run Tkinter main loop
root.mainloop()

# Release the capture after GUI is closed
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [2]:
import tkinter as tk
from tkinter import ttk
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image, ImageTk
import math

# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Initialize Tkinter GUI
root = tk.Tk()
root.title("Gesture-Based Radial Menu")
root.geometry("500x500")

# Canvas for radial menu
canvas = tk.Canvas(root, width=500, height=500, bg="white")
canvas.pack()

# Load icons or placeholders for radial menu buttons
icon_paths = [
    "elephant.png", "monkey.png", "tiger.png",
    "elephant.png", "monkey.png",  "tiger.png" ,"elephant.png", "monkey.png"
]
icons = [ImageTk.PhotoImage(Image.open(icon).resize((50, 50))) for icon in icon_paths]

# Radial menu settings
center_x, center_y = 250, 250  # Center of the radial menu
radius = 150  # Radius of the circle where icons will be placed
num_icons = len(icons)  # Number of icons to place

# Add icons in circular layout
buttons = []
for i, icon in enumerate(icons):
    angle = 2 * math.pi * i / num_icons  # Calculate angle for each icon
    icon_x = center_x + radius * math.cos(angle)
    icon_y = center_y + radius * math.sin(angle)
    # Place image as a button in the radial menu
    button = canvas.create_image(icon_x, icon_y, image=icon)
    buttons.append(button)

# Central volume control button or indicator
canvas.create_oval(center_x - 50, center_y - 50, center_x + 50, center_y + 50, fill="lightgrey")
volume_text = canvas.create_text(center_x, center_y, text="Volume\n62%", font=("Helvetica", 12))

# OpenCV Video Capture for Gesture Recognition
cap = cv2.VideoCapture(0)

# Gesture detection and interaction function
def detect_gesture():
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Get coordinates of thumb and index finger for pinch detection
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            
            # Calculate pixel coordinates
            thumb_coords = (int(thumb_tip.x * frame.shape[1]), int(thumb_tip.y * frame.shape[0]))
            index_coords = (int(index_tip.x * frame.shape[1]), int(index_tip.y * frame.shape[0]))

            # Distance between thumb and index to detect pinch
            distance = np.sqrt((thumb_coords[0] - index_coords[0]) ** 2 + (thumb_coords[1] - index_coords[1]) ** 2)
            
            # Define gesture actions based on distance and hand position
            if distance < 30:
                # Detect which radial button is closest to the hand position
                for i, button in enumerate(buttons):
                    button_coords = canvas.coords(button)
                    button_x, button_y = button_coords[0], button_coords[1]
                    # Calculate distance from hand to button
                    hand_to_button_distance = np.sqrt((button_x - index_coords[0]) ** 2 + (button_y - index_coords[1]) ** 2)
                    if hand_to_button_distance < 40:  # Trigger button click if hand is close enough
                        print(f"Button {i + 1} clicked!")  # Replace with actual action for each button

            # Use thumb y-coordinate to control volume (scroll effect)
            if thumb_coords[1] < frame.shape[0] // 3:
                canvas.itemconfig(volume_text, text="Volume\n70%")
            elif thumb_coords[1] > 2 * frame.shape[0] // 3:
                canvas.itemconfig(volume_text, text="Volume\n40%")
    
    # Display the frame
    cv2.imshow("Gesture Control", frame)
    cv2.waitKey(1)
    
    # Re-run this function after a short delay
    root.after(10, detect_gesture)

# Start gesture detection loop
root.after(10, detect_gesture)

# Run Tkinter main loop
root.mainloop()

# Release the capture after GUI is closed
cap.release()
cv2.destroyAllWindows()


TclError: image "pyimage3" doesn't exist

In [ ]:
import tkinter as tk
from tkinter import ttk
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image, ImageTk
import math

# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Initialize Tkinter GUI
root = tk.Tk()
root.title("Gesture-Based Radial Menu")
root.geometry("500x500")

# Canvas for radial menu
canvas = tk.Canvas(root, width=500, height=500, bg="white")
canvas.pack()

# Text options for radial menu buttons
options = [
    "Elephant", "Monkey", "Tiger",
    "Lion", "Giraffe", "Zebra",
    "Leopard", "Bear"
]

# Radial menu settings
center_x, center_y = 250, 250  # Center of the radial menu
radius = 150  # Radius of the circle where icons will be placed
num_options = len(options)  # Number of options to place

# Add text options in circular layout
buttons = []
for i, option in enumerate(options):
    angle = 2 * math.pi * i / num_options  # Calculate angle for each option
    option_x = center_x + radius * math.cos(angle)
    option_y = center_y + radius * math.sin(angle)
    # Place text in the radial menu
    button = canvas.create_text(option_x, option_y, text=option, font=("Helvetica", 12))
    buttons.append(button)

# Central volume control button or indicator
canvas.create_oval(center_x - 50, center_y - 50, center_x + 50, center_y + 50, fill="lightgrey")
volume_text = canvas.create_text(center_x, center_y, text="Volume\n62%", font=("Helvetica", 12))

# OpenCV Video Capture for Gesture Recognition
cap = cv2.VideoCapture(0)

# Gesture detection and interaction function
def detect_gesture():
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Get coordinates of thumb and index finger for pinch detection
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            
            # Calculate pixel coordinates
            thumb_coords = (int(thumb_tip.x * frame.shape[1]), int(thumb_tip.y * frame.shape[0]))
            index_coords = (int(index_tip.x * frame.shape[1]), int(index_tip.y * frame.shape[0]))

            # Distance between thumb and index to detect pinch
            distance = np.sqrt((thumb_coords[0] - index_coords[0]) ** 2 + (thumb_coords[1] - index_coords[1]) ** 2)
            
            # Define gesture actions based on distance and hand position
            if distance < 30:
                # Detect which radial button is closest to the hand position
                for i, button in enumerate(buttons):
                    button_coords = canvas.coords(button)
                    button_x, button_y = button_coords[0], button_coords[1]
                    # Calculate distance from hand to button
                    hand_to_button_distance = np.sqrt((button_x - index_coords[0]) ** 2 + (button_y - index_coords[1]) ** 2)
                    if hand_to_button_distance < 40:  # Trigger button click if hand is close enough
                        print(f"Option '{options[i]}' selected!")  # Replace with actual action for each option

            # Use thumb y-coordinate to control volume (scroll effect)
            if thumb_coords[1] < frame.shape[0] // 3:
                canvas.itemconfig(volume_text, text="Volume\n70%")
            elif thumb_coords[1] > 2 * frame.shape[0] // 3:
                canvas.itemconfig(volume_text, text="Volume\n40%")
    
    # Display the frame
    cv2.imshow("Gesture Control", frame)
    cv2.waitKey(1)
    
    # Re-run this function after a short delay
    root.after(10, detect_gesture)

# Start gesture detection loop
root.after(10, detect_gesture)

# Run Tkinter main loop
root.mainloop()

# Release the capture after GUI is closed
cap.release()
cv2.destroyAllWindows()


Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Elephant' selected!
Option 'Lion' selected!
Option 'Lion' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey' selected!
Option 'Monkey

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ragho\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\ragho\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\ragho\AppData\Local\Temp\ipykernel_2200\2153628370.py", line 56, in detect_gesture
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


In [1]:
import tkinter as tk
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image, ImageTk
import math

# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Initialize Tkinter GUI
root = tk.Tk()
root.title("Gesture-Based Radial Menu")
root.geometry("500x500")

# Canvas for radial menu
canvas = tk.Canvas(root, width=500, height=500, bg="white")
canvas.pack()

# Text options for radial menu buttons
options = [
    "Elephant", "Monkey", "Tiger",
    "Lion", "Giraffe", "Zebra",
    "Leopard", "Bear"
]

# Radial menu settings
center_x, center_y = 250, 250  # Center of the radial menu
radius = 150  # Radius of the circle where options will be placed
num_options = len(options)  # Number of options to place

# Add text options in circular layout
buttons = []
button_backgrounds = []  # Store background color rectangles for each option
for i, option in enumerate(options):
    angle = 2 * math.pi * i / num_options  # Calculate angle for each option
    option_x = center_x + radius * math.cos(angle)
    option_y = center_y + radius * math.sin(angle)
    
    # Background color rectangle (default white)
    rect = canvas.create_oval(option_x - 30, option_y - 30, option_x + 30, option_y + 30, fill="white", outline="")
    button_backgrounds.append(rect)
    
    # Text label for the radial menu option
    button = canvas.create_text(option_x, option_y, text=option, font=("Helvetica", 12))
    buttons.append(button)

# Central volume control button or indicator
canvas.create_oval(center_x - 50, center_y - 50, center_x + 50, center_y + 50, fill="lightgrey")
volume_text = canvas.create_text(center_x, center_y, text="Volume\n62%", font=("Helvetica", 12))

# OpenCV Video Capture for Gesture Recognition
cap = cv2.VideoCapture(0)

# Gesture detection and interaction function
def detect_gesture():
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    # Reset all button backgrounds to white
    for rect in button_backgrounds:
        canvas.itemconfig(rect, fill="white")
    
    if results.multi_hand_landmarks:
        for hand_index, hand_landmarks in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Get the index finger tip coordinates for each hand
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            index_coords = (int(index_tip.x * frame.shape[1]), int(index_tip.y * frame.shape[0]))

            # Map OpenCV coordinates to Tkinter canvas coordinates
            canvas_x = int(index_tip.x * 500)
            canvas_y = int(index_tip.y * 500)

            # Check if the index finger is hovering over any text option
            for i, button in enumerate(buttons):
                button_coords = canvas.coords(button)
                button_x, button_y = button_coords[0], button_coords[1]
                
                # Calculate distance from index finger to button center
                hand_to_button_distance = np.sqrt((button_x - canvas_x) ** 2 + (button_y - canvas_y) ** 2)
                
                if hand_to_button_distance < 30:  # Hovering distance threshold
                    # Change the background color of the section to blue
                    canvas.itemconfig(button_backgrounds[i], fill="blue")
                    print(f"Hovering over '{options[i]}'")  # Optional: Print which option is being hovered over

            # Use thumb y-coordinate to control volume (scroll effect)
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            thumb_coords = (int(thumb_tip.x * frame.shape[1]), int(thumb_tip.y * frame.shape[0]))
            
            if thumb_coords[1] < frame.shape[0] // 3:
                canvas.itemconfig(volume_text, text="Volume\n70%")
            elif thumb_coords[1] > 2 * frame.shape[0] // 3:
                canvas.itemconfig(volume_text, text="Volume\n40%")
    
    # Display the frame
    cv2.imshow("Gesture Control", frame)
    cv2.waitKey(1)
    
    # Re-run this function after a short delay
    root.after(10, detect_gesture)

# Start gesture detection loop
root.after(10, detect_gesture)

# Run Tkinter main loop
root.mainloop()

# Release the capture after GUI is closed
cap.release()
cv2.destroyAllWindows()


Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Zebra'
Hovering over 'Zebra'
Hovering over 'Zebra'
Hovering over 'Zebra'
Hovering over 'Zebra'
Hovering over 'Zebra'
Hovering over 'Zebra'
Hovering over 'Zebra'
Hovering over 'Zebra'
Hovering over 'Zebra'
Hovering over 'Zebra'
Hovering over 'Lion'
Hovering over 'Lion'
Hovering over 'Leopard'
Hovering over 'Leopard'
Hovering over 'Bear'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Giraffe'
Hovering over 'Elephant'
Hovering over 'Elephant'
Hovering over 'Elephant'
Hovering over 'Elephant'


In [3]:
import tkinter as tk
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image, ImageTk
import math

# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Initialize Tkinter GUI
root = tk.Tk()
root.title("Gesture-Based Radial Menu")
root.geometry("500x500")

# Canvas for radial menu
canvas = tk.Canvas(root, width=500, height=500, bg="white")
canvas.pack()

# Load cursor icon image
cursor_image = Image.open("cursor-icon.png")  # Replace with your icon path
cursor_icon = ImageTk.PhotoImage(cursor_image)

# Create a cursor icon on the canvas
cursor = canvas.create_image(-50, -50, image=cursor_icon)  # Start off-screen

# Text options for radial menu buttons
options = [
    "Elephant", "Monkey", "Tiger",
    "Lion", "Giraffe", "Zebra",
    "Leopard", "Bear"
]

# Radial menu settings
center_x, center_y = 250, 250  # Center of the radial menu
radius = 150  # Radius of the circle where options will be placed
num_options = len(options)  # Number of options to place

# Add text options in circular layout
buttons = []
button_backgrounds = []  # Store background color rectangles for each option
for i, option in enumerate(options):
    angle = 2 * math.pi * i / num_options  # Calculate angle for each option
    option_x = center_x + radius * math.cos(angle)
    option_y = center_y + radius * math.sin(angle)
    
    # Background color rectangle (default white)
    rect = canvas.create_oval(option_x - 30, option_y - 30, option_x + 30, option_y + 30, fill="white", outline="")
    button_backgrounds.append(rect)
    
    # Text label for the radial menu option
    button = canvas.create_text(option_x, option_y, text=option, font=("Helvetica", 12))
    buttons.append(button)

# Central volume control button or indicator
canvas.create_oval(center_x - 50, center_y - 50, center_x + 50, center_y + 50, fill="lightgrey")
volume_text = canvas.create_text(center_x, center_y, text="Volume\n62%", font=("Helvetica", 12))

# OpenCV Video Capture for Gesture Recognition
cap = cv2.VideoCapture(0)

# Gesture detection and interaction function
def detect_gesture():
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    # Reset all button backgrounds to white
    for rect in button_backgrounds:
        canvas.itemconfig(rect, fill="white")
    
    if results.multi_hand_landmarks:
        for hand_index, hand_landmarks in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Get the index finger tip coordinates for each hand
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            index_coords = (int(index_tip.x * frame.shape[1]), int(index_tip.y * frame.shape[0]))

            # Map OpenCV coordinates to Tkinter canvas coordinates
            canvas_x = int(index_tip.x * 500)
            canvas_y = int(index_tip.y * 500)

            # Update cursor position
            canvas.coords(cursor, canvas_x, canvas_y)

            # Check if the index finger is hovering over any text option
            for i, button in enumerate(buttons):
                button_coords = canvas.coords(button)
                button_x, button_y = button_coords[0], button_coords[1]
                
                # Calculate distance from index finger to button center
                hand_to_button_distance = np.sqrt((button_x - canvas_x) ** 2 + (button_y - canvas_y) ** 2)
                
                if hand_to_button_distance < 30:  # Hovering distance threshold
                    # Change the background color of the section to blue
                    canvas.itemconfig(button_backgrounds[i], fill="blue")
                    print(f"Hovering over '{options[i]}'")  # Optional: Print which option is being hovered over

            # Use thumb y-coordinate to control volume (scroll effect)
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            thumb_coords = (int(thumb_tip.x * frame.shape[1]), int(thumb_tip.y * frame.shape[0]))
            
            if thumb_coords[1] < frame.shape[0] // 3:
                canvas.itemconfig(volume_text, text="Volume\n70%")
            elif thumb_coords[1] > 2 * frame.shape[0] // 3:
                canvas.itemconfig(volume_text, text="Volume\n40%")
    
    # Display the frame
    cv2.imshow("Gesture Control", frame)
    cv2.waitKey(1)
    
    # Re-run this function after a short delay
    root.after(10, detect_gesture)

# Start gesture detection loop
root.after(10, detect_gesture)

# Run Tkinter main loop
root.mainloop()

# Release the capture after GUI is closed
cap.release()
cv2.destroyAllWindows()


TclError: image "pyimage1" doesn't exist